In [ ]:
import numpy as np

# Aufgabe 3: Gauß-Algorithmus

Programmieren Sie in Python eine Routine `x = pivotgauss(A,b)`, welche für eine vorgegebene reguläre Matrix $A \in \mathbb{R}^{n\times n}$ und rechte Seite $b \in \mathbb{R}^n$ die Lösung $x \in \mathbb{R}^n$ der Gleichung $Ax = b$ liefert. Erfolgen soll dies mit dem Gauß-Verfahren mit Spalten-Pivotsuche.
 
> **Latest Tips:**
> * Das Ergebnis eines Vektor-Ausschnitts (z.B. `v[2:]`) ist ein neuer Vektor passender Länge, eine Kopie des Original-Ausschnitts. Wie bei jedem Vektor beginnen die Indizes der verkuerzten Kopie mit 0.
> * Zu Vergleichs- oder Testzwecken ist die Nutzung von Numpy-Lösungsroutinen, z.B. aus `numpy.linalg`, zulässig. Ihre Gauß-Implementierung soll aber ohne diese auskommen.
> * Stellen Sie sicher, dass Sie auf die richtigen Elemente der Matrix A zugreifen, nämlich durch `A[row, col]`.
> * Für die Fehlersuche: `nan` steht für Not A Number und `inf` für infinite. Diese zeigen sehr häufig Programmfehler an, meist eine Division durch Null, weshalb man deren Auftreten gründlich überprüfen sollte.
> * Sie dürfen zusätzlich `np.argmax` verwenden. Beachten Sie die Python-Online-Hilfe, um die Routine gut zu verstehen.
> * Numpy erlaubt negative Indizes; `v[-1]` ist der letzte Eintrag; `v[-2]` der vorletzte usw.

Fügen Sie hier Ihre Funktion pivotgauss ein:

In [ ]:
def pivotgauss(A, b):
    """
    Gauß-Elimination mit Spalten-Pivotsuche, damit Loesen des
    linearen Gleichungssystems  A*x=b .
    Die Eingangsparameter sind numpy-Arrays, der Loesungsvektor x ist
    als ebensolches anzulegen.
    ACHTUNG: Werden Matrizen als Adresse oder als Wert uebergeben?
    Bei 'call-by-reference' (sehr wahrscheinlich!) veraendern Zuweisungen
    in der Routine die Eintraege der globalen Matrix! Darum wird hier
    mit Kopien von A und b gearbeitet.
    """
    A = np.copy(A)
    x = np.copy(b)
    n = x.size
    for i in range(n - 1):
        # Pivotelement finden
        pivot_index = (
            np.argmax(np.abs(A[i:, i])) + i
        )  # 'pivot_index' muss bei 'i' beginnen
        # Falsche optionen:
        # pivot_index = np.argmax(A[i:, i]) + i
        # pivot_index = np.argmax(A[i:, i])

        # Wenn nötig Zeilen tauschen
        if pivot_index != i:
            A[[i, pivot_index], i:] = A[[pivot_index, i], i:]
            x[i], x[pivot_index] = x[pivot_index], x[i]

        # Gauß-Elimination
        pivot_value = A[i, i]
        if pivot_value != 0.0:
            ratio = A[i + 1 :, i] / pivot_value
            A[i + 1 :, i:] -= np.outer(ratio, A[i, i:])
            x[i + 1 :] -= x[i] * ratio
    x[n - 1] /= A[n - 1, n - 1]
    for i in range(n - 2, -1, -1):
        x[i] = (x[i] - np.inner(A[i, (i + 1) :], x[(i + 1) :])) / A[i, i]
    return x

## Testskript


Das Testskript testet Ihren Gauß-Algorithmus (mit Pivotsuche) mit
verschiedenen Matrizen:

* einer $32 \times 32$ Matrix, die aus zwei Dreiecksmatrizen zusammengesetzt ist ($A=L*R$)
* einer negativen $8 \times 8$-Hilbert-Matrix mit Nullen in der letzten Zeile und Spalte
  ausser im letzten Diagonalelement

und (falls beide ersten Tests erfolgreich waren)

* einer Reihe von $(n,n)$-Hilbert-Matrizen mit $n = 2,4,8,16,32$.



In [ ]:
correct = True

### 1. Testlauf mit einer (32,32)-Matrix

In [ ]:
n = 32
mat = np.zeros((n, n))
counter = 1
for i in range(n):
    mat[i, i:] = np.arange(counter, counter + n - i)
    counter = counter + n - i
mat_reverse = np.zeros((n, n))
counter = 1
for i in range(n - 1, -1, -1):
    mat_reverse[i, 0 : (i + 1)] = np.arange(counter + i, counter - 1, -1)
    counter = counter + i + 1
A = mat @ mat_reverse
A[[3, 5, 1, 26, 17, 19, 14, 0], :] = A[[0, 1, 3, 5, 14, 17, 19, 26], :]
b = np.sum(A, axis=1)
A, b

In [ ]:
x = pivotgauss(A, b)
reference = np.ones(32)
error = np.linalg.norm(reference - x)
error

In [ ]:
if np.any(np.isnan(error)) or not np.all(np.isfinite(error)) or error > 1e-9:
    correct = True
    raise AssertionError(f"Die exakte Lösung ist {reference}, Ihre Lösung lautet {x}.")

### 2. Testlauf mit modifizierter Hilbert-Matrix

In [ ]:
n = 8
A = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        A[i, j] = 1.0 / (i + j + 1)
A = -A
A[-1, :-1] = 0.0
A[:-1, -1] = 0.0
b = np.sum(A, axis=1)
A, b

In [ ]:
x = pivotgauss(A, b)
reference = np.ones(n)
error = np.linalg.norm(reference - x)
error

In [ ]:
if np.any(np.isnan(error)) or not np.all(np.isfinite(error)) or error > 1e-6:
    correct = True
    raise AssertionError(f"Die exakte Lösung ist {reference}, Ihre Lösung lautet {x}.")

In [ ]:
if correct:
    print("Gratulation: Ihr Gauss-Pivot-Algorithmus scheint korrekt zu sein!")

### 3. Testlauf mit Hilbert-Matrizen der Größe $n=2,4,8,16,32$

In [ ]:
for n in [2, 4, 8, 16, 32, 64, 128]:
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            A[i, j] = 1.0 / (i + j + 1)
    b = np.sum(A, axis=1)
    x = pivotgauss(A, b)
    reference = np.ones(n)
    error = np.linalg.norm(reference - x)
    print(f"Größe {n} -> Fehler {error}")